# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
%matplotlib inline

# Load data and Prepare data


In [2]:
df=pd.read_csv('data/Iris.csv')
df=df.drop("Id",axis=1)
df=df.rename(columns={"species":'labels'})

In [3]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,labels
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Train-Test split

In [4]:
def train_test_split(df,test_size):
    if isinstance(test_size,float):
        test_size=round(test_size*len(df))
    
    indices = df.index.tolist()
    test_indices = random.sample(population=indices,k=test_size)
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    
    return train_df,test_df

In [5]:
random.seed(0)
train_df,test_df = train_test_split(df,test_size=20)

In [6]:
train_df.head()

,sepal_length,sepal_width,petal_length,petal_width,labels
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Check purity??

In [7]:
def check_purity(data):
    label_col = data[:,-1]
    unique_classes = np.unique(label_col)
    
    if len(unique_classes)==1:
        return True
    else:
        return False

## Classify

In [8]:
def classify(data):
    label_col = data[:,-1]
    unique_classes,counts_unique_classes = np.unique(label_col,return_counts=True)
    index=counts_unique_classes.argmax()
    classification = unique_classes[index]
    return classification

### Potential splits

In [9]:
def get_potential_splits(data):
    potential_splits={}
    _,n_cols = data.shape
    for column_index in range(n_cols-1):
        potential_splits[column_index]=[]
        values = data[:,column_index]
        unique_values = np.unique(values)
        
        for index in range(len(unique_values)):
            if index!=0:
                current_value = unique_values[index]
                prev_value = unique_values[index-1]
                potential_split = (current_value+prev_value)/2
                potential_splits[column_index].append(potential_split)
    return potential_splits

In [10]:
def split_data(data,split_column,split_value):
    split_column_values = data[:,split_column]
   
    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values<=split_value]
        data_above = data[split_column_values > split_value]
        
    else:
        data_below = data[split_column_values==split_value]
        data_above = data[split_column_values!=split_values]
        
    return data_above,data_below

### Entropy

In [11]:
def calculate_entropy(data):
    label_column=data[:,-1]
    _,counts = np.unique(label_column,return_counts=True)
    probabilities = counts/counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [12]:
def calculate_overall_entropy(data_below,data_above):
    n = len(data_below)+len(data_above)
    p_data_below = len(data_below)/n
    p_data_above = len(data_above)/n
    
    overall_entropy = (p_data_below * calculate_entropy(data_below))+(
    p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

### Determining best split

In [13]:
def determine_best_split(data,potential_splits):
    overall_entropy = 9999
    
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below,data_above = split_data(data,column_index,value)
            current_overall_entropy = calculate_overall_entropy(data_below,data_above)
            
            if current_overall_entropy < overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    return best_split_column,best_split_value
            

### determine type of feature

In [14]:
def determine_feature_type(df):
    feature_types=[]
    n_unique_values_threshold =15
    for feature in df.columns:
        if feature!="label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]
            
            
            if (isinstance(example_value,str)) or (len(unique_values)<n_unique_values_threshold):
                feature_types.append('categorical')
            else:
                feature_types.append('continuous')
    return feature_types
    

### Algorithm

In [15]:
def decision_tree_algorithm(df,counter=0,min_samples=2,max_depth=5):
    if counter == 0:
        global COLUMN_HEADERS,FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_feature_type(df)
        data = df.values
    else:
        data=df
    
    
    if (check_purity(data) or (len(data)<min_samples)) or (counter == max_depth):
        classification = classify(data)
        
        return classification
    
    else:
        counter+=1
        
        
        potential_splits = get_potential_splits(data)
        split_column,split_value = determine_best_split(data,potential_splits)
        data_above,data_below = split_data(data,split_column,split_value)
        
        
        if len(data_below)==0 or len(data_above)==0:
            classification = classify(data)
            return classifiaction
        
        
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        
        if type_of_feature == 'continuous':
            question ="{} <= {}".format(feature_name,split_value)
            
        else:
            question = "{}={}".format(feature_name,split_value)
        
        
        sub_tree ={question :[]}
        
        
        yes_answer = decision_tree_algorithm(data_below,counter,min_samples,max_depth)
        no_answer = decision_tree_algorithm(data_above,counter,min_samples,max_depth)
        
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
            
            
        return sub_tree

In [18]:
tree = decision_tree_algorithm(train_df, max_depth=3)

tree


{'petal_length <= 2.5999999999999996': ['Iris-setosa',
  {'petal_width <= 1.65': [{'petal_length <= 4.95': ['Iris-versicolor',
      'Iris-virginica']},
    'Iris-virginica']}]}

In [29]:
train_df.columns
test_df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'labels',
       'classification'],
      dtype='object')

In [25]:


def predict(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    if not isinstance(answer, dict):
        return answer
    else:
        residual_tree = answer
        return predict(example, residual_tree)




In [26]:
def accuracy(df, tree):
    df["classification"] = df.apply(predict, args=(tree,), axis=1)
    df["classification_correct"] = df["classification"] == df["label"]
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [28]:
score = accuracy(test_df, tree)
score

KeyError: 'label'